# Guide to Unique Decoding of Reed-Solomon Codes

This notebook provides the explanation and implementation of the **Welch-Berlekamp algorithm** for the unique decoding of Reed-Solomon codes. We will start by defining the unique decoding problem, develop the intuition behind the algorithm using a geometric perspective, and finally build and demonstrate a working Python implementation.

## The Unique Decoding Problem

A Reed-Solomon code is created by taking a message, treating it as the coefficients of a **message polynomial** $P(X)$ of degree less than $k$, and then evaluating that polynomial at $n$ distinct points $(\alpha_1, \dots, \alpha_n)$ to create a codeword.

When this codeword is transmitted, a noisy channel may introduce errors, resulting in a **received word** $y = (y_1, \dots, y_n)$. The number of errors, $e$, is the number of positions where the received word differs from the original codeword.

The goal of **unique decoding** is to recover the *one and only* original polynomial $P(X)$ from the noisy received word $y$. This is only guaranteed to be possible if the number of errors is strictly less than half the minimum distance of the code. For a Reed-Solomon code, this condition is:

$$ e < \frac{n-k+1}{2} $$

This notebook will build the Welch-Berlekamp algorithm, an efficient method for solving this exact problem.


## A Geometric View of the Problem 

To better visualize the problem, we can perform a "syntactic shift" and think of the received word $y$ not as a vector, but as a collection of $n$ points in a 2D plane: $\{(\alpha_1, y_1), (\alpha_2, y_2), \dots, (\alpha_n, y_n)\}$.

<img src="./imgs/image_unique_decoding_1.png" width="600"/>

The image above shows an example of a received word with n=14 points and k=2. The original message was a line (a polynomial of degree k-1=1).

The original, uncorrupted codeword would consist of points that all lie perfectly on the curve defined by the message polynomial $P(X)$. The effect of noise is to knock some of these points off the curve. The decoder's job, in this geometric view, is to find the unique curve of degree less than $k$ that passes through the maximum number of these received points.

<img src="./imgs/image_unique_decoding_2.png"  width="600"/>

The above image illustrates this. The correct polynomial, $P(X)=X$, passes through the subset of "correct" points, while the "error" points lie scattered off the line.

## The Core Idea: Reverse Engineering a Solution 

The Welch-Berlekamp algorithm is designed using a "reverse engineering" approach. We start by assuming we magically know the solution—both the original polynomial $P(X)$ and the locations of the errors—and derive a mathematical property. Then, we use that property to build an algorithm that finds $P(X)$ without knowing the error locations beforehand.

### The Error-Locator Polynomial, E(X)

Let's define a special tool called the **Error-Locator Polynomial**, $E(X)$. This is a polynomial whose roots are the x-coordinates ($\alpha_i$) where an error occurred. In other words:

$$ E(\alpha_i) = 0 \quad \text{if} \quad y_i \neq P(\alpha_i) $$

If there are $e$ errors, we can construct such a polynomial of degree $e$.

### The Key Equation

With this definition, we can establish a key equation that holds true for **every single point**, whether it's an error or not:

$$ y_i E(\alpha_i) = P(\alpha_i) E(\alpha_i) \quad \text{for all } i=1, \dots, n $$


This powerful identity is easy to prove by considering two cases:
1.  **If an error occurred at $\alpha_i$**: By definition, $E(\alpha_i) = 0$. The equation becomes $y_i \cdot 0 = P(\alpha_i) \cdot 0$, which simplifies to $0=0$. The identity holds.
2.  **If no error occurred at $\alpha_i$**: In this case, we know $y_i = P(\alpha_i)$. If we multiply both sides of this by $E(\alpha_i)$, the equality is preserved. The identity also holds.

This equation is the foundation of the algorithm. The only problem is that it involves the product of two unknowns, $P(X)$ and $E(X)$, making it a difficult quadratic problem to solve directly.

## 3. The Core Idea: Reverse Engineering a Solution 

The Welch-Berlekamp algorithm is designed using a clever "reverse engineering" approach. We start by assuming we magically know the solution—both the original polynomial $P(X)$ and the locations of the errors—and derive a mathematical property. Then, we use that property to build an algorithm that finds $P(X)$ without knowing the error locations beforehand.

### The Error-Locator Polynomial, E(X)

Let's define a special tool called the **Error-Locator Polynomial**, $E(X)$. This is a polynomial whose roots are the x-coordinates ($\alpha_i$) where an error occurred. In other words:

$$ E(\alpha_i) = 0 \quad \text{if} \quad y_i \neq P(\alpha_i) $$

If there are $e$ errors, we can construct such a polynomial of degree $e$.

### The Key Equation

With this definition, we can establish a key equation that holds true for **every single point**, whether it's an error or not:

$$ y_i E(\alpha_i) = P(\alpha_i) E(\alpha_i) \quad \text{for all } i=1, \dots, n $$


This powerful identity is easy to prove by considering two cases:
1.  **If an error occurred at $\alpha_i$**: By definition, $E(\alpha_i) = 0$. The equation becomes $y_i \cdot 0 = P(\alpha_i) \cdot 0$, which simplifies to $0=0$. The identity holds.
2.  **If no error occurred at $\alpha_i$**: In this case, we know $y_i = P(\alpha_i)$. If we multiply both sides of this by $E(\alpha_i)$, the equality is preserved. The identity also holds.

This equation is the foundation of the algorithm. The only problem is that it involves the product of two unknowns, $P(X)$ and $E(X)$, making it a difficult quadratic problem to solve directly.